In [91]:
from database import *

def ticks_to_kbars(ticks, interval='5Min'):

    kbars = pd.DataFrame()
    
    kbars['open'] = ticks['close'].resample(interval).first()
    kbars['close'] = ticks['close'].resample(interval).last()
    kbars['high'] = ticks['close'].resample(interval).max()
    kbars['low'] = ticks['close'].resample(interval).min()
    kbars['volume'] = ticks['volume'].resample(interval).sum()
    
    kbars.dropna(inplace=True)
    
    return kbars

In [92]:
import talib

def get_technical_indicator(kbars):
    
#     kbars['rsi'] = talib.RSI(kbars.close, timeperiod=14)
    kbars['cci'] = talib.CCI(kbars.high, kbars.low, kbars.close, timeperiod=14)
    
    macd, macdsignal, macdhist = talib.MACD(kbars.close, fastperiod=5, slowperiod=34, signalperiod=5)
    
    kbars['macd'] = macd
    kbars['macdsignal'] = macdsignal
    kbars['macdhist'] = macdhist
    
    kbars['slowk'], kbars['slowd'] = talib.STOCH(kbars.high,
                        kbars.low,
                        kbars.close,
                        fastk_period=9,
                        slowk_period=3,
                        slowk_matype=0,
                        slowd_period=3,
                        slowd_matype=0)
    
    return kbars

In [93]:
import json

with open('config.json') as f:
  config = json.load(f)

api = sj.Shioaji()
api.login(
    person_id = config['account'],
    passwd= config['password'],
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done."))

# api.logout()

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.


Exception ignored in: <function Shioaji.__del__ at 0x7f94f2395310>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/shioaji/shioaji.py", line 652, in __del__
    self._solace.__del__()
  File "shioaji/backend/solace/api.pyx", line 1232, in shioaji.backend.solace.api.SolaceAPI.__del__
  File "shioaji/backend/solace/api.pyx", line 327, in shioaji.backend.solace.api.SolaceAPI.logout
  File "/opt/anaconda3/lib/python3.8/site-packages/shioaji/utils.py", line 49, in raise_resp_error
    raise Exception(resp)
Exception: {}


<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.


In [97]:
code = '2330'

tw_calendar = get_calendar('XTAI')

date = pd.to_datetime('2021-1-7')
prev_trading_date = tw_calendar.previous_close(date).date()

ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
kbars = ticks_to_kbars(ticks)
kbars = get_technical_indicator(kbars)
# kbars = kbars[pd.to_datetime(prev_trading_date).replace(hour=13, minute=30):]
kbars = kbars[date:]

kbars

,open,close,high,low,volume,cci,macd,macdsignal,macdhist,slowk,slowd
ts,,,,,,,,,,,
2021-01-07 09:00:00,554.0,556.0,556.0,553.0,4229,2.556962e+02,1.748017,-0.459258,2.207275,90.476190,65.079365
2021-01-07 09:05:00,556.0,556.0,559.0,555.0,2289,2.275000e+02,3.245321,0.775602,2.469719,88.988095,82.043651
2021-01-07 09:10:00,557.0,558.0,558.0,556.0,865,1.754349e+02,4.677048,2.076084,2.600964,91.666667,90.376984
2021-01-07 09:15:00,558.0,558.0,559.0,557.0,731,1.373243e+02,5.487904,3.213357,2.274547,89.583333,90.079365
2021-01-07 09:20:00,558.0,557.0,559.0,557.0,1451,1.038555e+02,5.616864,4.014526,1.602337,91.666667,90.972222
2021-01-07 09:25:00,557.0,556.0,558.0,556.0,446,7.481884e+01,5.314745,4.447932,0.866813,87.500000,89.583333
2021-01-07 09:30:00,557.0,555.0,557.0,555.0,671,5.314900e+01,4.747419,4.547761,0.199657,76.250000,85.138889
2021-01-07 09:35:00,555.0,556.0,556.0,555.0,630,4.622642e+01,4.576577,4.557366,0.019210,70.416667,78.055556
2021-01-07 09:40:00,556.0,556.0,557.0,556.0,404,4.867493e+01,4.382017,4.498917,-0.116899,60.000000,68.888889


In [5]:
def day_trading_backtest(code, date, connection, api):
    
    tw_calendar = get_calendar('XTAI')
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    kbars = ticks_to_kbars(ticks)
    kbars = get_technical_indicator(kbars)
    kbars = kbars[date:]
            
    entry_price = 0
    entry_time = None
    
    exit_price = 0
    exit_time = None
    
    position = 0
            
    for ts in range(len(kbars)):
        
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=5)
        print(' start current_time = {}'.format(current_time))
        current_price = kbars['close'][ts]
        print('current_price = {}'.format(current_price))
#         macdhist = kbars.iloc[ts]['macdhist']
#         print('macdhist = {}'.format(macdhist))
#         cci = kbars.iloc[ts]['cci']
#         print('cci = {}'.format(cci))
#         print('end')
        
        if (
#             kbars.iloc[ts - 3:ts]['rsi'].min() < 30 and
#             kbars.iloc[ts - 1]['macdhist'] < 0 and
            current_time < date.replace(hour=13, minute=20, second=0) and
            kbars.iloc[ts]['cci'] > -100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position == 0
        ):
            
            position = 1          
            entry_price = current_price
            entry_time = current_time.time()

            print('[{}] buy {} at {}'.format(current_time, code, current_price))
        
        elif (
#             kbars.iloc[ts - 3:ts]['rsi'].max() > 70 and
#             kbars.iloc[ts - 1]['macdhist'] > 0 and
            kbars.iloc[ts]['cci'] > 100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position != 0
        ):
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
        elif (
            current_time >= date.replace(hour=13, minute=20, second=0) and
            position != 0
        ):
            
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                     code, 
                                     entry_time,
                                     entry_price, 
                                     position * 1000,
                                     entry_price * position * 1000,
                                     exit_time,
                                     exit_price,
                                     position * 1000,
                                     exit_price * position * 1000]
                                   ],
                                    columns=[
                                        '成交日期', 
                                        '股票代號',
                                        '買進時間',
                                        '買進價格',
                                        '買進股數',
                                        '買進金額',
                                        '賣出時間',
                                        '賣出價格',
                                        '賣出股數',
                                        '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()

In [6]:
day_trading_backtest('2456', pd.to_datetime('2021/3/2'), connection, api)

<SecurityType.Option: 'OPT'> fetch done.
 start current_time = 2021-03-02 09:05:00
current_price = 114.0
[2021-03-02 09:05:00] buy 2456 at 114.0
 start current_time = 2021-03-02 09:10:00
current_price = 112.5
[2021-03-02 09:10:00] sell 2456 at 112.5


,成交日期,股票代號,買進時間,買進價格,買進股數,買進金額,賣出時間,賣出價格,賣出股數,賣出金額
0,2021-03-02,2456,09:05:00,114.0,1000,114000.0,09:10:00,112.5,1000,112500.0
